In [ ]:
EVENT_HUB_CONN_STR="Endpoint=sb://factored-datathon.servicebus.windows.net/;SharedAccessKeyName=datathon_listener;SharedAccessKey=sJJnyi8GGTBAa55jY89kacoT6hXAzWx2B+AEhCPEKYE=;EntityPath=factored_datathon_amazon_review"
EVENTHUB_NAME="factored-datathon"

In [ ]:
import os
from azure.eventhub import EventHubConsumerClient


consumer_client = EventHubConsumerClient.from_connection_string(
    conn_str=EVENT_HUB_CONN_STR,
    consumer_group='$Default',
    eventhub_name=EVENTHUB_NAME,
)

with consumer_client:
    # consumer_client is now ready to be used.
    consumer_client.get_eventhub_properties()

In [ ]:
import logging
from azure.eventhub import EventHubConsumerClient

client = EventHubConsumerClient.from_connection_string(
    conn_str=EVENT_HUB_CONN_STR,
    consumer_group='$Default',
    eventhub_name=EVENTHUB_NAME,
)

logger = logging.getLogger("azure.eventhub")
logging.basicConfig(level=logging.INFO)

def on_event(partition_context, event):
    logger.info("Received event from partition {}".format(partition_context.partition_id))
    partition_context.update_checkpoint(event)
    print("Received event from partition {}".format(partition_context.partition_id))

with client:
    client.receive(
        on_event=on_event,
        starting_position="-1",  # "-1" is from the beginning of the partition.
    )
    # receive events from specified partition:
    # client.receive(on_event=on_event, partition_id='0')

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import logging
import asyncio
from azure.eventhub.aio import EventHubConsumerClient

logger = logging.getLogger("azure.eventhub")
logging.basicConfig(level=logging.INFO)

async def on_event(partition_context, event):
    logger.info("Received event from partition {}".format(partition_context.partition_id))
    print("Received event from partition {}".format(partition_context.partition_id))
    await partition_context.update_checkpoint(event)

async def receive():
    client = consumer_client = EventHubConsumerClient.from_connection_string(conn_str=EVENT_HUB_CONN_STR,consumer_group='$Default',eventhub_name=EVENTHUB_NAME,)
    async with client:
        await client.receive(
            on_event=on_event,
            starting_position="-1",  # "-1" is from the beginning of the partition.
        )
        # receive events from specified partition:
        # await client.receive(on_event=on_event, partition_id='0')

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(receive())